# REMIND Electricity Sector Biosphere Flows

Calculate all biosphere flow coefficients for the REMIND electricity sector. **Values are given per kWh**.

In [5]:
%run initialize_notebook.ipynb

In [9]:
%run REMIND_Functions_to_modify_ecoinvent.ipynb

Geomatcher: Used 'AU' for 'AUS'
Geomatcher: Used 'CA' for 'CAN'
Geomatcher: Used 'HM' for 'HMD'
Geomatcher: Used 'NZ' for 'NZL'
Geomatcher: Used 'PM' for 'SPM'
Geomatcher: Used 'CN' for 'CHN'
Geomatcher: Used 'HK' for 'HKG'
Geomatcher: Used 'MO' for 'MAC'
Geomatcher: Used 'TW' for 'TWN'
Geomatcher: Used 'AX' for 'ALA'
Geomatcher: Used 'AT' for 'AUT'
Geomatcher: Used 'BE' for 'BEL'
Geomatcher: Used 'BG' for 'BGR'
Geomatcher: Used 'CY' for 'CYP'
Geomatcher: Used 'CZ' for 'CZE'
Geomatcher: Used 'DE' for 'DEU'
Geomatcher: Used 'DK' for 'DNK'
Geomatcher: Used 'ES' for 'ESP'
Geomatcher: Used 'EE' for 'EST'
Geomatcher: Used 'FI' for 'FIN'
Geomatcher: Used 'FR' for 'FRA'
Geomatcher: Used 'FO' for 'FRO'
Geomatcher: Used 'GB' for 'GBR'
Geomatcher: Used 'GI' for 'GIB'
Geomatcher: Used 'GR' for 'GRC'
Geomatcher: Used 'HR' for 'HRV'
Geomatcher: Used 'HU' for 'HUN'
Geomatcher: Used 'IM' for 'IMN'
Geomatcher: Used 'IE' for 'IRL'
Geomatcher: Used 'IT' for 'ITA'
Geomatcher: Used 'LT' for 'LTU'
Geomatch

In [3]:
%run EI2REMIND_Interface_Helpers.ipynb

In [8]:
databases

Databases dictionary with 9 object(s):
	Carma CCS
	biosphere3
	ecoinvent_3.5
	ecoinvent_Remind_BAU_2030
	ecoinvent_Remind_BAU_2050
	ecoinvent_Remind_RCP26_2030
	ecoinvent_Remind_RCP26_2050
	ecoinvent_Remind_RCP37_2030
	ecoinvent_Remind_RCP37_2050

In [41]:
def rmnd_bioflows(region, scenario="BAU", year=2030, tech_primers={}):
    """ Extract bioflows for the REMIND electricity sector in a REMIND region.
        A dict with primers {x: y} can be given to specify ecoinvent technology y to represent
        REMIND technology x.
    """
    carma_techs = ['Biomass IGCC CCS', 'Biomass IGCC', 'Coal IGCC', 'Coal IGCC CCS', 'Coal PC CCS', 'Gas CCS']

    eidb_name = get_REMIND_database_name(scenario, year)
    eidb = Database(eidb_name)
    carmadb_name = import_karma(eidb_name=eidb_name)
    carmadb = Database(carmadb_name)

    result = {}
    bio_names = []
    regions = ei_locations_in_remind_region(region)
    print("Found following regions for {}: {}".format(region, regions))
    for key in available_electricity_generating_technologies:
        print("Processing RMND Technology {}".format(key))
        if key in tech_primers:
            techs = [tech_primers[key]]
        else:
            techs = available_electricity_generating_technologies[key]
        if len(techs) == 0:
            print("No technologies available for {}! Skipping.".format(key))
            continue
        units = {
            "kilowatt hour": 1.,
            "megajoule": 1./3.6
        }
        if key in carma_techs:
            lca = lca_for_multiple_techs_and_regions(
                techs, regions, carmadb, units_and_conversions=units)
        else:
            lca = lca_for_multiple_techs_and_regions(
                techs, regions, eidb, units_and_conversions=units)

        # bionames are only generated once. This takes some time.
        # probably these never change...
        if len(bio_names) != lca.inventory.shape[0]:
            print("Bionames changed, old length: {}".format(len(bio_names)))
            bio_names = [get_activity(key)["name"] for key in lca.biosphere_dict]
            print("new length: {}".format(len(bio_names)))

        # flows are aggregated by the resp. material
        # we do not care *where* the material flows to
        result[key] = pd.DataFrame.from_dict({
            "flow": bio_names,
            "amount": [lca.inventory[n, :].sum() for n in range(len(bio_names))]
        }).groupby("flow").agg({"amount": sum})

    return pd.concat(result)

In [42]:
df = rmnd_bioflows("EUR")

Extracted 1 worksheets in 0.36 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.30 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 11/16/2018 16:21:05
  Finished: 11/16/2018 16:21:06
  Total time elapsed: 00:00:00
  CPU %: 62.60
  Memory %: 3.00
Created database: Carma CCS
Found following regions for EUR: ['EUR', 'SE', 'FI', 'GR', 'GB', 'AX', 'HR', 'IT', 'ES', 'DE', 'PT', 'DK', 'FR', 'IE', 'BALTSO', 'FO', 'EE', 'NL', 'Canary Islands', 'CENTREL', 'CY', 'MT', 'IM', 'LT', 'AT', 'BE', 'BG', 'CZ', 'GI', 'HU', 'LU', 'LV', 'PL', 'RO', 'SI', 'SK', 'RER']
Processing RMND Technology Biomass IGCC CCS
Bionames changed, old length: 0
new length: 2079
Processing RMND Technology Biomass IGCC
Processing RMND Technology Coal IGCC
Processing RMND Technology Coal IGCC CCS
Processing RMND Technology Coal PC CCS
Processing RMND Technology Gas CCS
Processing RMND Technology Biomass CHP
Processing RMND Technology Coal PC
Processing RMND Technology Coal CHP
Processing RMND Technology Gas OC
Processing RMND Technology Gas CC
Processing RMND Technology Gas CHP
Processing RMND Techno

In [48]:
df.sort_index()[:30]

amount
            flow                                               
Biomass CHP 1,3-Dioxolan-2-one                     5.510340e-10
            1,4-Butanediol                         6.388529e-12
            1-Pentanol                             6.019678e-13
            1-Pentene                              7.955420e-12
            2,2,4-Trimethyl pentane                4.339695e-15
            2,4-D                                  6.614058e-09
            2,4-D amines                           7.536979e-20
            2,4-D ester                            5.005970e-20
            2,4-DB                                 4.998338e-20
            2,4-di-tert-butylphenol                0.000000e+00
            2-Aminopropanol                        5.777228e-13
            2-Methyl pentane                       6.968185e-10
            2-Methyl-1-propanol                    1.777725e-12
            2-Methyl-2-butene                      3.218249e-16
            2-Nitrobenzoic acid                    3.029699e-13
            2-Propanol                             1.673137e-08
            2-chlorobenzaldehyde                   0.000000e+00
            3-Methyl-1-butanol                     0.000000e+00
            4-Methyl-2-pentanol                    1.242667e-19
            4-Methyl-2-pentanone                   1.746576e-12
            AOX, Adsorbable Organic Halogen as Cl  4.020910e-09
            Abamectin                              0.000000e+00
            Acenaphthene                           1.437487e-12
            Acenaphthylene                         1.246118e-13
            Acephate                               1.572887e-10
            Acetaldehyde                           3.589300e-07
            Acetamide                              2.311345e-11
            Acetamiprid                            0.000000e+00
            Acetic acid                            5.075994e-08
            Acetochlor                             1.163104e-13